In [1]:
import pandas as pd
import numpy as np

In [6]:
#Data yang didapatkan di Kaggle pada akhir tahun 2019

df = pd.read_csv('20191226-reviews.csv')
df.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [16]:
df.dropna(inplace=True)
df[df['rating'] >= 3]
df['helpfulVotes'] = np.where(df['rating'] > 3, 5.0, 17.0)
df.head(10)

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,17.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",True,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",17.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",True,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,5.0
6,B0000SX2UC,the cell phone store owner,5,"April 16, 2004",True,Wanna cool Nokia? You have it here!,Cool. Cheap. Color: 3 words that describe the ...,5.0
7,B0000SX2UC,Matt,4,"April 3, 2004",True,Problem with 3588i universal headset,"The 3599i is overall a nice phone, except that...",5.0
8,B0000SX2UC,Charles Cook,5,"November 24, 2003",True,cool phone!!!!!!!!,"I've never owned a Nokia phone before, so this...",5.0
9,B0000SX2UC,Amazon Customer,3,"February 2, 2004",False,Pissed off-a little bit,ok well im in school and i need the text messa...,17.0
10,B0000SX2UC,habblie,4,"December 25, 2004",True,"works great, but don't dropt it",I've had this phone for over a year and I real...,5.0


In [17]:
df['helpfulVotes'].mean()

9.836874218922297

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['body'], df['helpfulVotes'], random_state = 0)

In [22]:
print('X_train first entry: \n\n', X_train[0])
print('\n\nX_train shape: ', X_train.shape)

X_train first entry: 

 I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing. It finally died on me so I used this Nokia phone I bought in a garage sale for $1. I wonder y she sold it so cheap?... Bad: ===> I hate the menu. It takes forever to get to what you want because you have to scroll endlessly. Usually phones have numbered categories so u can simply press the # and get where you want to go. ===> It's a pain to put it on silent or vibrate. If you're in class and it rings, you have to turn it off immediately. There's no fast way to silence the damn thing. Always remember to put it on silent! I learned that the hard way. ===> It's so true about the case. It's a mission to get off and will break ur nails in the process. Also, you'll damage the case each time u try. For some reason the phone started giving me problems once I did succeed in opening it. ===> Buttons could be a bit bigger. Vibration could be str

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer().fit(X_train)

In [24]:
vect.get_feature_names()[::3000]

['00',
 'autocoodrecte',
 'costó',
 'faded',
 'información',
 'n960u',
 'quieren',
 'sotware',
 'uuuggghhh']

In [25]:
len(vect.get_feature_names())

25386

In [26]:
X_train_vectorized = vect.transform(X_train)
X_train_vectorized

<20404x25386 sparse matrix of type '<class 'numpy.int64'>'
	with 1118414 stored elements in Compressed Sparse Row format>

In [28]:
#Pengujian dengan Logistic Regression

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

/home/gunturd/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
from sklearn.metrics import roc_auc_score

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.8755662746648314


In [33]:
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
print('Coef Smallest Helpful: \n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Coef Greatest Helpful: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))


Coef Smallest Helpful: 
['great' 'love' 'best' 'amazing' 'excellent' 'perfect' 'awesome'
 'perfectly' 'easy' 'far']

Coef Greatest Helpful: 
['not' 'disappointed' 'terrible' 'returned' 'return' 'returning' 'refund'
 'cannot' 'worst' 'worse']



In [31]:
#Mencoba menggunakan TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df = 5).fit(X_train)
len(vect.get_feature_names())


8172

In [32]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.879770823174786


In [34]:

feature_names = np.array(vect.get_feature_names())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print('Tfidf Smallest Helpful: \n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Tfidf Greatest Helpful: \n{}\n'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Tfidf Smallest Helpful: 
['impacts' 'forego' 'soup' 'balked' 'evaluators' 'laaaaag' 'preordering'
 'magicleap' 'directv' 'paradigm']

Tfidf Greatest Helpful: 
['price' 'reliability' 'me' 'refurbished' 'stars' 'gracias' 'worked'
 'works' 'exelente' 'nada']



In [35]:
#Uji Coba pada beberapa isu, ambil dari data

print(model.predict(vect.transform(['Great phone service and options, lousy case!', 
                                   'This phone gave me a concussion. and may come with a faulty battery.'])))

[ 5. 17.]
